# Análisis del riesgo de incumplimiento de los prestatarios

Este proyecto  esta diriguido diriguido a la division de prestamos de un banco. Con el objetivo de veriguar si el numero de hijos, estado civil, o estatus familia tienen alguna relacion con el incumplimiento del pago a tiempo de un prestamo. gracias a que el banco tienen algunos datos sobre el historial crediticio de algunos clientes, vamos a proceder a analizarlos.



Esperando que sea de gran utilidad para la correcta calificasion de cada uno de los clientes, procederemos a realizar un analisis minucioso y limpio, para lograr excelentes resultados al momento de tomar esta desicion tan importante.

## Abre el archivo de datos y mira la información general. 



In [1]:
import pandas as pd

Trabajaremos con rutas absolutas para una mejor lectura de los datos, pero dejaremos el archivo a analizar en este repositorio, para que lo puedan leer u obtner la ruta absoluta como lo hice yo.

In [114]:
credit_scoring = pd.read_csv('credit_scoring_eng.csv')

In [ ]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


## Ejercicio 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

Realizamos un vista de la informacion de mis datos en donde se denote el numero de filas, columnas y con que tipo de variables voy a trabajar, y hay un metodo que es muy eficiente para saber ese tipo de informacion.

In [ ]:
credit_scoring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Observamos la informacion general de los datos, nos damos cuenta de que es un DataFreme bastante grande,
sin embargo tambien notamos valores ausentes en algunas columnas, asi que vamos a mostrarlos

In [ ]:
credit_scoring.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Observamos que los valores ausentes en las columnas ['days_employed'] y ['total_income'] son los mismos, lo que nos lleva a considerar que sus valores ausentes son simultáneos.


Vamos a realizar algunas agrupaciones entre estas columnas para verificar el comportamiento de sus valores nulos.

In [ ]:
credit_scoring[['days_employed','total_income']].isna().value_counts()

days_employed  total_income
False          False           19351
True           True             2174
Name: count, dtype: int64

Una vez realizada esta agrupacion y tomando en cuanta que los valores simultaneos existen unicamente en los valores nulos
de las columnas de dias laborados e ingreso mensual de clientes. Pero tambien tenemos que deterinar que tanta influencia tienen estos valores nulos siultaneos en estas columnas.

In [ ]:
display(credit_scoring[(credit_scoring['days_employed'].isnull()) & (credit_scoring['total_income'].isnull())]) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Hay que tomar en cuenta que los valores asentes simultaneos en estas columnas tienen 2 cosas en comun. La primera, los clientes que la mayoria de clientes son de edad avanzada,y la segunda es que su tipo de ingreso mensual esta denotado a partir del restiro o de un negocio.

Esto nos lleva a pensar que los valores ausentes, son simultaneos en estas dos columnas unicamnete, y resulta sensato pensar que la falta de valores puede deberse a los datos que especificamos anteriormente, dadas las condiciones que observamos en otras columnas. Ahora pasaremos a analizar cada una de las columnas por separado.

 Para empezar vamos ha revisar cada una de las columnas por separado para fijarnos como se ven, y modificarlas si es necesario para poder trabajar con ellas de mejor manera empezando con la columna de dias trabajados ['days_employed'].

In [ ]:
credit_scoring['days_employed']

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64

Obtenemos los valores ausentes de la columna ['days_employed'] para poder relacionarlas con diferentes columnas y denotar patrones de silimitud o diferencia.

Primero  realizaremos una descripcion de los datos en generales para tomar en cuenta aspectos importantes en esta columna como medianas o medias y ver si nos podria ser de utilidad en el futuro.

In [ ]:
credit_scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


Existen valores ausentes  solo en algunas columnas de DataFrame,tenemos buenas medianas y minimos, vamos a buscar valores ausentes de la columna 'days_employed' filtrandola  en diferentes columnas para ver averiguar si existe algun patron en los valores que ausentes de ciertas condiciones.

In [ ]:
credit_scoring[credit_scoring['days_employed'].isna()].groupby('income_type')['income_type'].count()

income_type
business          508
civil servant     147
employee         1105
entrepreneur        1
retiree           413
Name: income_type, dtype: int64

Existen valores ausentes en diferentes condiciones y columnas, el obejetivo seria averiguar que tienen en comun esas condiciones de los clientes.

In [ ]:
credit_scoring[credit_scoring['days_employed'].isna()].groupby(credit_scoring['education'])['education'].count()

education
BACHELOR'S DEGREE        23
Bachelor's Degree        25
PRIMARY EDUCATION         1
Primary Education         1
SECONDARY EDUCATION      67
SOME COLLEGE              7
Secondary Education      65
Some College              7
bachelor's degree       496
primary education        19
secondary education    1408
some college             55
Name: education, dtype: int64

Agrupando esta columna por el nivel de educasion, lo primero a tomar en cunta es que la mayoria de valores ausentes se encuentan en los clientes cuya educasion es secundaria en su mayoria.

In [ ]:
credit_scoring[credit_scoring['days_employed'].isna()].groupby('family_status')['family_status'].count()

family_status
civil partnership     442
divorced              112
married              1237
unmarried             288
widow / widower        95
Name: family_status, dtype: int64

Realizando la misma agrupacion de valores bausentes por la columna de estatus de familia, Observamos que los clientes que son casados tiene una cantidad considerable de valores ausentes. 

In [ ]:
credit_scoring[credit_scoring['days_employed'].isna()].groupby('children')['children'].count()

children
-1        3
 0     1439
 1      475
 2      204
 3       36
 4        7
 5        1
 20       9
Name: children, dtype: int64

En esta columna, el mayor problema a tomar en cuenta es que existe un numero negativo de cantidad de de hijos y que tampoco es muy probable que haya una persona con 20 hijos, quizas tengamos que solucionarlo en el futuro.Lo importante en este apartado es manifestar que existe una mayor cantidad de valores ausentes en las personas que no tienen hijos.

In [ ]:
credit_scoring[credit_scoring['days_employed'].isna()].groupby('gender')['gender'].count()

gender
F    1484
M     690
Name: gender, dtype: int64

En esta columna es sensato pensar que si existen mayor cantiad de valores ausentes en clientes que son mujeres,
deberia ser debido a que en su mayoria los clientes son mujeres 

**Conclusión intermedia**

los valores ausentes difieren dependiendo la condicion del cliente. Los clientes que estan casados, no tienen hijos y que no han tenido ninguna deuda anterior son los que tienen la mayor cantidad de valores ausentes. Se trata de un numero considerable de valores ausentes en clientes que cumplen con ciertas condiciones asi que vamos a ver si vale la pena completarlos como en su mayoria se encuentra en las condiciones y columnas ['children']  ['dob_years'] y ['family_status'].

Vamos a seguir averiguando valores ausentes en estas posibles columnas para confirmar si existe algun patron para calificarlos.

In [ ]:
credit_scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


una ves relizadas las conversiones necesarias y algunos filtros, podemos notar que tenemos valores ausentes  segun ciertas condiciones al agrupar y filtrar varias columnas con ['days_employed'] vamos a indagar mas sobre estas condiciones, y en que columnas son similares los valores ausentes para  encontrar algun paron importante

**Posibles razones por las que hay valores ausentes en los datos**

Hasta el momento la mayoria de valores ausentes se encuentan en las condiciones de que un ciente sea casado ['merried'], su tipo de trabajo sea empleado ['employed'], y que no tenga hijos ['children] = 0. Asi que se podria decir que existe un patron en el hecho de que falten muchos valores justamente en esas condiciones, vamos a seguir indagando posibles razones externas, ya que tambien pueden ser datos aleatorios

Empecemos a comprobar si los valores ausentes son aleatorios.

In [ ]:
credit_scoring.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

comprobamos los tipos de datos que tenemos, para verificar cual de ellos necesitan una transformacion

In [ ]:
credit_scoring['days_employed'].astype('float32')

0         -8437.672852
1         -4024.803711
2         -5623.422852
3         -4124.747070
4        340266.062500
             ...      
21520     -4529.316895
21521    343937.406250
21522     -2113.346924
21523     -3112.481689
21524     -1984.507568
Name: days_employed, Length: 21525, dtype: float32

Transformamos la columna de dias empleados a numeros de coma flotante para poder seguir trabjando con ellos y sobre todo para poder realizar operaciones

In [ ]:
credit_scoring['dob_years'].astype('float64')

0        42.0
1        36.0
2        33.0
3        32.0
4        53.0
         ... 
21520    43.0
21521    67.0
21522    38.0
21523    38.0
21524    40.0
Name: dob_years, Length: 21525, dtype: float64

De la misma manera transformamos la columna de la edad de los clientes a numeros de coma flotante para realizar calculos mas exactos proximamente.

**Conclusión intermedia**

La distribucion de la tabla original es diferente a la tabla filtrada lo cual significa que los valores ausentes representan una parte importantel de DataFrame.

Sin embargo aún no podemos llegar a ninguna conclusión, investiguemos más a fondo nuestro conjunto de datos. Pensemos en otras razones que podrían llevar a la ausencia de datos y verifiquemos si podemos encontrar algún patrón que nos haga pensar que los valores ausentes no son aleatorios.

Vamos verificar cada una de las columnas por seprado para comprobar que tan veridico es este postulado.


Ahora vamos a ver la columna de la edad de los clientes ['dob_years]

In [ ]:
credit_scoring['dob_years']

0        42
1        36
2        33
3        32
4        53
         ..
21520    43
21521    67
21522    38
21523    38
21524    40
Name: dob_years, Length: 21525, dtype: int64

En este caso hay demasiados clientes con edades diferentes, asi que vamos a crear una funcion que agrupe a los clientes por un grupo de edad especificado, y reflejar datos mas concretos respecto a un grupo de clientes en lugar de reflejar a cada uno con su respectiva edad.

In [ ]:
def dob_years_group(dob_years):
    if dob_years < 30:
        return '20-30'
    if dob_years < 40:
        return '30-39'
    if dob_years < 50:
        return '40-49'
    if dob_years < 60:
        return '50-59'
    else:
            return '+60'

In [ ]:
credit_scoring['dob_years'].apply(dob_years_group).value_counts()

dob_years
30-39    5674
40-49    5371
50-59    4679
20-30    3284
+60      2517
Name: count, dtype: int64

In [ ]:
credit_scoring['dob_years_group'] = credit_scoring['dob_years'].apply(dob_years_group)

A travez de una funcion, creamos una nueva columna ['dob_years_group'] para crear grupos en la columna de 'dob_years' y hacerla mas pequeña para relacionarlas con otras columnas.

In [ ]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59


Una vez creada nuestra columna, procedemos a relacionar medias y medianas con otras columnas para obtener resultados mas generales

In [ ]:
credit_scoring['total_income'].isna().groupby(credit_scoring['dob_years_group']).mean()

dob_years_group
+60      0.104092
20-30    0.094093
30-39    0.099577
40-49    0.099981
50-59    0.107074
Name: total_income, dtype: float64

Obtenemos una estadistica resumida de medias en los valores ausentes de la columna ['total_income'] y la columna ['dob_years_group'],
para ver si hay relacion entres sus valores ausentes, y en su mayoria los valores ausentes se ecuentan en promedio, en las personas que tienen mayor rango de edad.

**Conclusión intermedia**

en ciertas condiciones existen una gran catidad de valores ausentes y en otras la cantidad de valores ausentes es igual vamos a realizar unas ultimas relaciones para concluir si estamos tratando con valores ausentes que son accidentales, o si siguen un patron que nos demuestre una relacion especifica entre ellos.

In [ ]:
credit_scoring.groupby([credit_scoring['dob_years_group'], credit_scoring['education']])['total_income'].mean()

dob_years_group  education          
+60              BACHELOR'S DEGREE      33635.665611
                 Bachelor's Degree      35171.049368
                 Graduate Degree        15800.399000
                 PRIMARY EDUCATION      14549.736600
                 Primary Education      15816.533000
                                            ...     
50-59            bachelor's degree      33773.000236
                 graduate degree        42945.794000
                 primary education      17770.700978
                 secondary education    24029.137548
                 some college           27880.563689
Name: total_income, Length: 65, dtype: float64

Esta relacion es mas grande  ya que ustilizamos 3 columnas para, obtener un resultado general que nos indica que los clientes
que son mayores y tienen un mayor nivel educativo, manejan un mayor nivel de ingreso mesual que aquellos clientes que son mas jovenes o solo han llegado hasta la secundaria.

In [ ]:
credit_scoring.groupby([credit_scoring['dob_years_group'], credit_scoring['education']])['days_employed'].mean()

dob_years_group  education          
+60              BACHELOR'S DEGREE      303170.872935
                 Bachelor's Degree      129813.468069
                 Graduate Degree        337584.815560
                 PRIMARY EDUCATION      357647.000025
                 Primary Education      375860.167488
                                            ...      
50-59            bachelor's degree      102138.384674
                 graduate degree         -5352.038180
                 primary education      200652.244032
                 secondary education    134981.465135
                 some college           113423.863514
Name: days_employed, Length: 65, dtype: float64

De la misma manera observamos que las peronas que tienen mayor edad, cuentan en promedio con mayor
cantidad de dias laborados que aquellas personas que son mas jovenes y tienen un mayor nivel educativo.

**Conclusiones**

Los clientes ,cuya educacion llego a la secundaria, estan casados, y que no tienen hijos, son los que tienen la mayoria de valores ausentes debido a que, agrupando diferentes colmunas por los dias laborados hayamos relacion de valores ausentes en una gran proporcion 

seria sensato relenar los valores ausentes utilizando las medias de las columnas de las cuales tenemos datos mas sensatos. 

en la siguiente parte vamos a rrelenar valores ausentes y descubrir o borrar duplicados de ser necesario, tambien vamos a cambiar la ortografia de ciertas columnas si es que se considera necesario con el objetivo de tener una tabla mas limpia para poder trabajar.

## Transformación de datos

Vamos Repasar cada columna para ver qué problemas podemos tener en ellas

En perimera instancia vamos a pasar a minusculas la columna que nos indica la educaion del cliente para poder ver sus duplicados reales.

In [ ]:
credit_scoring['education']

0          bachelor's degree
1        secondary education
2        Secondary Education
3        secondary education
4        secondary education
                ...         
21520    secondary education
21521    secondary education
21522    secondary education
21523    secondary education
21524    secondary education
Name: education, Length: 21525, dtype: object

Notamos que en la columna ['education'], existen datos que tienen mayusculas y minusculas.Vamos a aplicar un .str.lower() transformarlas a minusculas para observar sus datos duplicados

In [ ]:
credit_scoring['education'] = credit_scoring['education'].str.lower()
credit_scoring['education']

0          bachelor's degree
1        secondary education
2        secondary education
3        secondary education
4        secondary education
                ...         
21520    secondary education
21521    secondary education
21522    secondary education
21523    secondary education
21524    secondary education
Name: education, Length: 21525, dtype: object

Realizamos una descripcion general de nuestra columna para observar de mejor manera como se encuentra distribuida.

In [ ]:
credit_scoring.children.describe()

count    21525.000000
mean         0.538908
std          1.381587
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

In [ ]:
credit_scoring.groupby('family_status')['children'].mean().sort_values()

family_status
widow / widower      0.223958
unmarried            0.286527
divorced             0.457741
civil partnership    0.510175
married              0.638207
Name: children, dtype: float64

Relacionamos la columna 'family_status' con 'children' y notamos que las personas casadas o divorciadas son las que tienen  datos relevantes sobre la cantiad de hijos.

In [ ]:
credit_scoring.groupby('children')['children'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Es muy poco probable que exista un cliente con -1 hijos o por el contrario con 20 hijos, asi que realizaremos un calculo de etas dos variables para obtener datos mas realistas.

In [ ]:
credit_scoring['children'] = credit_scoring['children'].abs()


Pasamos la columna a valor absoluto, para manejar valores positivos como tal y hacer mas entendible la columna  para realizar nuestro conteo, Sin afectar nuestros datos originales.

In [ ]:
credit_scoring.groupby('children')['children'].count()

children
0     14149
1      4865
2      2055
3       330
4        41
5         9
20       76
Name: children, dtype: int64

En el conteo de esta columna observamos que los datos en numero de hijos ['children'] que eran negativos ya no aparecen negativos.

Ahora vamos ha averiguar que tan probable es que un cliente haya tenido 20 hijos.

In [ ]:
credit_scoring.loc[credit_scoring['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group
606,20,-880.221113,21,secondary education,1,married,0,M,business,0,23253.578,purchase of the house,20-30
720,20,-855.595512,44,secondary education,1,married,0,F,business,0,18079.798,buy real estate,40-49
1074,20,-3310.411598,56,secondary education,1,married,0,F,employee,1,36722.966,getting an education,50-59
2510,20,-2714.161249,59,bachelor's degree,0,widow / widower,2,F,employee,0,42315.974,transactions with commercial real estate,50-59
2941,20,-2161.591519,0,secondary education,1,married,0,F,employee,0,31958.391,to buy a car,20-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,secondary education,1,married,0,F,employee,1,21363.842,to own a car,40-49
21325,20,-601.174883,37,secondary education,1,married,0,F,business,0,16477.771,profile education,30-39
21390,20,NaN,53,secondary education,1,married,0,M,business,0,NaN,buy residential real estate,50-59
21404,20,-494.788448,52,secondary education,1,married,0,M,business,0,25060.749,transactions with my real estate,50-59


Es muy poco realista que existan personas con esta cantidad de hijos ya que no tenemos personas con 6,7,8, etc... hijos.
Sin embargo tampoco podemos pasar por alto el hecho de que la mayoria tiene un estatus familiar 'casado' o algo que haga referencia a que tuvo una pareja.

In [ ]:
credit_scoring['children'] = credit_scoring['children'].replace({20:2})

In [ ]:
credit_scoring.groupby('children')['children'].count()

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Realizando un conteo de esta columna, creada a partir de la observamos que ya no hay personas con 20 hijos, por que reemplazamos el valor 20 por el valor 2. Esto deberia solucionar el problema de que un cliente tenga una cantidad tan exesiva de hijos.

In [ ]:
credit_scoring['children'].describe()

count    21525.000000
mean         0.479721
std          0.755528
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64

Realizamos algunas peuqeñas modificasiones en esta columna especificamnete, para que ['children'] cuente con valores mas prudentes a tomar en cuenta para calculos y relaciones proximas.

En la columna [`days_employed`]. En primer lugar, vamos a trasnformar a positivos y realizaremos un pequeño calculo para que nos arroje datos mas realistas de los dias laborados de un cliente.

In [ ]:
credit_scoring['days_employed']

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64

como no es prudente eliminar las filas con datos problematicos, ya que afectaria nuestro resultado, vamos a reslizar a lgunos calculos por de tiempo de los datos que tenemos. Por ejemplo podriamos dvidirlos por el numero de dias del año o multiplicarlos por 24 horas.El objetivo es que  queremos datos posistivos asi que con una transformacion a valor absoluto de la columna bederia ser suficiente

In [ ]:
credit_scoring['days_employed'] = abs(credit_scoring['days_employed'])
credit_scoring['days_employed']

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

In [ ]:
credit_scoring.loc[credit_scoring['days_employed'] > 36500, 'days_employed'] =  credit_scoring.loc[credit_scoring['days_employed'] > 36500, 'days_employed']/24
credit_scoring['days_employed']

0         8437.673028
1         4024.803754
2         5623.422610
3         4124.747207
4        14177.753002
             ...     
21520     4529.316663
21521    14330.725172
21522     2113.346888
21523     3112.481705
21524     1984.507589
Name: days_employed, Length: 21525, dtype: float64

En cuanto a la edad de los clientes, lo primero a tomar en cuenta es que seria complicado analizar cada edad individual de los clientes ya que existen muchas edades, asi que utilizaremos mas adelante una funcion que nos ayude a crear grupos que conprendan rangos de edad, para realizar un analisis mas detallado.

In [ ]:
credit_scoring['dob_years']

0        42
1        36
2        33
3        32
4        53
         ..
21520    43
21521    67
21522    38
21523    38
21524    40
Name: dob_years, Length: 21525, dtype: int64

tenemos un numero muy grande datos en esta columna, asi que crearecmos una columna con los diferentes grupos de edad que nos facilite encontrar patrones de categoria.

In [ ]:
def dob_years_group(dob_years):
    if dob_years < 30:
        return '20-30'
    if dob_years < 40:
        return '30-39'
    if dob_years < 50:
        return '40-49'
    if dob_years < 60:
        return '50-59'
    else:
            return '+60'

In [ ]:
credit_scoring['dob_years'].apply(dob_years_group).value_counts()

dob_years
30-39    5674
40-49    5371
50-59    4679
20-30    3284
+60      2517
Name: count, dtype: int64

In [ ]:
credit_scoring['dob_years_group'] = credit_scoring['dob_years'].apply(dob_years_group)

In [ ]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,14177.753002,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59


En cuanto a la edad de un cliente. Seria logico relacionarla con la columna de los dias laborados ['days_employed'] para verificar el promedio de nuestros datos en esas columnas, y ver si nos pueden ser e utilidad para completar aquellos valores que son iguales a 0.

In [ ]:
credit_scoring.groupby('dob_years_group')['days_employed'].mean()

dob_years_group
+60      12743.739504
20-30     1346.485726
30-39     2103.700591
40-49     3065.967424
50-59     7541.593571
Name: days_employed, dtype: float64

Podriamos utilizar estos promedios de la columna ['days_employed'] para completar los valores que son iguales a 0. A travez de una funcion.

In [ ]:
def fill_by_group(row):
    if (row['days_employed'] > 0) and (row['days_employed'] <= 20):
        return 1346.485726
    elif (row['days_employed'] > 20) and (row['days_employed'] <= 30):
        return 2103.700591
    elif (row['days_employed'] > 30) and (row['days_employed'] <= 40):
        return 3065.967424
    elif (row['days_employed'] > 40) and (row['days_employed'] <= 50):
        return 7541.593571
    else:
        
        return row['days_employed']

In [ ]:
credit_scoring.loc[credit_scoring['days_employed'].isna(), 'days_employed'] = credit_scoring.loc[credit_scoring['days_employed'].isna()].apply(fill_by_group, axis=1)

In [ ]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,14177.753002,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59


Una vez que utilizamos nuestra funcion obetnemos una tabla en la que los valores asunetes de ['days_employed'] son rellenados con el promedio de la columna de grupo de edad.

Ahora revisemos la columna [`family_status`], para ver que tipo de valores nos arroja y que tipo de errores podriamos abordar.

In [ ]:
credit_scoring.family_status.describe()

count       21525
unique          5
top       married
freq        12380
Name: family_status, dtype: object

In [ ]:
credit_scoring['family_status'].unique()

array(['married', 'civil partnership', 'widow / widower', 'divorced',
       'unmarried'], dtype=object)

Revisamos los valores unicos en string de esta columna y observamos que es una cantidad  poco relevante de este tipo de valores.

In [ ]:
credit_scoring['family_status']

0                  married
1                  married
2                  married
3                  married
4        civil partnership
               ...        
21520    civil partnership
21521              married
21522    civil partnership
21523              married
21524              married
Name: family_status, Length: 21525, dtype: object

En esta columna, en sus valores unicos no representan una cantidad importante ya que no existen 2 categorias que se parecan y que que deban ser iguales como tal. Para asegurarnos de esto realizaremos una agrupacion final de la columna.

In [ ]:
credit_scoring.groupby('family_status')['family_status'].count()

family_status
civil partnership     4177
divorced              1195
married              12380
unmarried             2813
widow / widower        960
Name: family_status, dtype: int64

Bajo esta agrupacion, Observamos que en su mayoria, los clientes son personas casadas y que existen muy pocos clientes
que son biudos o biudas.

Ahora revisaremos la columna `gender`. realizando un descripcion para ver cuales son los puntos mas importantes de la esta columna

In [ ]:
credit_scoring.gender.describe()

count     21525
unique        3
top           F
freq      14236
Name: gender, dtype: object

Al observar la descripcion de la columna notamos que en su mayoria los clientes son mujeres, existen muy pocos valores unicos y no hay valores ausentes en la columna, aun asi, obtenemos un conteo bastante amplio de los datos de esta columna.

Vamos a verificar que tanta importacia tienen estos 3 valores unicos en esta columna, ya que es una columna que solo deberia tener dos valores tipo str de genero que son 'F' mujer, y 'M' hombres.

In [ ]:
credit_scoring['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Al obtener los valores unicos de esta columna tenemos una anomalia en el valor 'XNA', pero antes de considerar su eliminacion, observaremos la frecuencia relativa dentro de los datos de esta columna.

In [ ]:
credit_scoring['gender'].value_counts(normalize=True)

gender
F      0.661370
M      0.338583
XNA    0.000046
Name: proportion, dtype: float64

A observar este resultado, notamos que el valor 'XNA' no representa un numero importante de datos para esta columna, Asi que podria considerarse la idea de eliminar este valor unico en especifico.

In [ ]:
credit_scoring = credit_scoring[credit_scoring['gender'] != 'XNA']

A travez de este codigo vamos a eliminar aquellas filas  donde el valor en la columna ['gender'] sea igual a 'XNA'. Ya que no representa una influencia relevante en esta columna

In [ ]:
credit_scoring.groupby('gender')['gender'].count()

gender
F    14236
M     7288
Name: gender, dtype: int64

Una vez eliminado el valor unico 'XNA' de esta columna, realizamos un conteo general de nuestros dotos en esta columna y efectivamente, valores ya no se encuentran y el hecho de que en su mayoria los clientes son mujeres, cada vez tiene mas relevancia.

Ahora vamos a revisar la columna `income_type`. Veremos qué tipo de valores hay y qué problemas podemos abordar, comenzando por una descripcion  general de la columna.

In [ ]:
credit_scoring.income_type.describe()

count        21524
unique           8
top       employee
freq         11119
Name: income_type, dtype: object

Segun la descripcion de esta columna manejamos una frecuencia bastante amplia de los datos sin embargo, tambien tomemos en cuenta que dentro de esta frecuencia exiten 8 valores unicos, asi que vamos a mostarlos para verificar que tan relevantes son en esta columna.

In [ ]:
credit_scoring['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

Al denotar nuestros valores unicos de la columna ['income_type'] contamos con 8 valores, los cuales no parecen tener tanta relevancia toamando en cuenta la cantidad. Pero antes de considerar eliminarlos, realizaremos unos conteos y sacaremos algunos promedios de esta columna, relacioandola con otras del mismo indole.

In [ ]:
credit_scoring.groupby('income_type')['total_income'].mean().sort_values()

income_type
paternity / maternity leave     8612.661000
student                        15712.260000
unemployed                     21014.360500
retiree                        21940.394503
employee                       25820.841683
civil servant                  27343.729582
business                       32386.741818
entrepreneur                   79866.103000
Name: total_income, dtype: float64

Al  agrupar por  el ingreso mensual ['income_type'] por el total de ingresos mesnuales ['total_income'], pues nos damos cuenta que aquellas personas que tienen negocios o son emprendedores tienen mayores ingresos que otras categorias de tipo de ingreso.

In [ ]:
credit_scoring = credit_scoring[credit_scoring['income_type'] != 'paternity / maternity leave']
credit_scoring = credit_scoring[credit_scoring['income_type'] != 'unemployed']

Realizamos la eliminacion de solo 2 valores unicos que en  que en este caso no representan mayor relevancia. Sin embargo los valores restantes pueden ser clientes que si representen valores importantes en el resto de las columnas.

In [ ]:
credit_scoring['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'entrepreneur',
       'student'], dtype=object)

Mostramos de nuevo nuestros valores unicos para comprovar que se haya realizado la modificasion necesaria.

In [ ]:
credit_scoring.groupby('income_type')['income_type'].count()

income_type
business          5084
civil servant     1459
employee         11119
entrepreneur         2
retiree           3856
student              1
Name: income_type, dtype: int64

Mediante un conteo general de la columna, observamos que existe un solo estudiante, que puede ser de utilidad tomar en cuenta para una proxima puntuacion crediticia en el futuro, cuando pase al grupo de los empleados, ya que como mayoria los clientes  que arrojan mayor cantidad de valores son empleados.

Vamos a ver los valores duplicados en nuestro conjunto de datos para comprobar si es una cantidad importante que pueda influir en nuestro resultado final.

In [ ]:
credit_scoring.duplicated().sum()

71

Al hacer una suma de nustros duplicados observamos que de acuerdo a la cantidad de datos totales que tenemos,no parece haber un  numero importante, pero vamos  ha hacer algunas pruebas  para verificar nuestros datos.

In [ ]:
credit_scoring.drop_duplicates()   

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,14177.753002,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-49
21521,0,14330.725172,67,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,+60
21522,1,2113.346888,38,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,30-39
21523,3,3112.481705,38,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,30-39


In [ ]:
credit_scoring = credit_scoring.drop_duplicates()

Guardamos los resultados y volvemos realizar la suma de valores duplicados.

In [ ]:
credit_scoring.duplicated().sum()

0

Despues de eliminar los datos duplicados vamos ha realizar un conteo de los datos para ver si hay diferemncia con el conjunto original de datos.

In [ ]:
credit_scoring.count()

children            21450
days_employed       19347
dob_years           21450
education           21450
education_id        21450
family_status       21450
family_status_id    21450
gender              21450
income_type         21450
debt                21450
total_income        19347
purpose             21450
dob_years_group     21450
dtype: int64

Al deshasernos de los duplicados observamos que no hay una diferencia importante con respecto a nuestro conjunto original de datos. Con este conjunto tambien podemos llegar a una conclucion acertada ya que en este caso, los valores duplicados no representan un acantidad importante.

# Trabajar con valores ausentes

En ese apartado vamos  primeramente a mostrar los diccionarios que muestran las columnas ['family_satus'], y ['education'] ya que representan ids que son importantes para estas columnas por que son los que nos dan el resultado de categoria para cada una de ellas

In [ ]:
credit_scoring.set_index('education_id')['education'].to_dict()

{0: "bachelor's degree",
 1: 'secondary education',
 2: 'some college',
 3: 'primary education',
 4: 'graduate degree'}

Encontramos los dicsionarios en la columna ['education'] ya que proporciona un ID's de clasidicacion. Ahora continuamos con los diccionarios de la columna [ family_status'].

In [ ]:
credit_scoring[['family_status_id', 'family_status']].value_counts(sort=False).index

MultiIndex([(0,           'married'),
            (1, 'civil partnership'),
            (2,   'widow / widower'),
            (3,          'divorced'),
            (4,         'unmarried')],
           names=['family_status_id', 'family_status'])

Al igual que con la columna anterior, la columna que nos da el estatus de familia ['family_status'] tambien cuenta con una columna que califica los datos de la columna y es prudente tomar en cuenta ambas columnas para encontrar y trabajar los valores ausentes.

### Restaurar valores ausentes en `total_income`

Las columnas que manejan una catidad mas relevante de valores ausentes son ['total_income'] primeramente, vamos a rellenarlos a travez de una funcion, aunque no vamos a descartar la posibilidad de rellenarlos con los valores medios o medianos de la columna.

In [ ]:
credit_scoring.groupby('income_type')['total_income'].median()

income_type
business         27571.0825
civil servant    24071.6695
employee         22815.1035
entrepreneur     79866.1030
retiree          18962.3180
student          15712.2600
Name: total_income, dtype: float64

observamos que dentro de esta agrupacion los clientes cuyo tipo de ingreso se determina a partir de un emprendimiento o  son retirados, tienen mayor cantidad de ingresos en promedio,sin embargo no tenemos tantos clientes que sean emprendedores. Asi que puede ser prudente rellenar los valores ausentes de ['total_income'] con esta mediana.

In [ ]:
def ausent_sust (row):
    edid = row['education_id']
    total = row['total_income']
    if total =="":
        if edid ==0:
            x=27571.0825
        if edid ==1:
            x=24071.6695
        if edid ==2:
            x=22815.1035
        if edid ==3:
            x=79866.1030
        if edid ==4:
            x=18962.3180
            row['total_income'] = x
    return row
credit_scoring['ausent_sust'] = credit_scoring.apply(ausent_sust,axis=1)['total_income']

In [ ]:
credit_scoring['ausent_sust']

0        40620.102
1        17932.802
2        23341.752
3        42820.568
4        25378.572
           ...    
21520    35966.698
21521    24959.969
21522    14347.610
21523    39054.888
21524    13127.587
Name: ausent_sust, Length: 21450, dtype: float64

Agregamos este resultado a cada una de las filas que manejen valores nulos y comprobamos que nustra columna origianl no presenta muchas diferencia , luego de hacer esta modificasion. Sin embargo vamos a realizar algunos promedios adicionales para verificar esto.

En primera instancia, realizaremos un promedio, estadistico verificando de medianas a travez de una descripcion general de nuestra tabla completa.

In [ ]:
credit_scoring.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,ausent_sust
count,21450.000000,19347.000000,21450.000000,21450.000000,21450.000000,21450.000000,19347.000000,19347.000000
mean,0.480513,4640.730272,43.272821,0.817063,0.973986,0.081072,26788.802865,26788.802865
std,0.756046,5355.380393,12.570987,0.548635,1.421669,0.272952,16476.068349,16476.068349
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,3306.762000,3306.762000
25%,0.000000,926.925347,33.000000,1.000000,0.000000,0.000000,16492.834000,16492.834000
50%,0.000000,2194.207271,42.000000,1.000000,0.000000,0.000000,23202.870000,23202.870000
75%,1.000000,5535.883055,53.000000,1.000000,1.000000,0.000000,32549.611000,32549.611000
max,5.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,362496.645000,362496.645000


En promedio tenemos menor cantidad de datos en la columna de dias laborados ['days_employed'], y tambien observamos que tenemos
la misma catidad de datos en ['total_income'] y ['ausent_sust'], columna creada  a base de promedios del tipo de ingreso ['income_type].

In [ ]:
credit_scoring.groupby('total_income')['days_employed'].mean().sort_values()

total_income
26712.386       24.141633
19858.460       24.240695
37033.790       30.195337
20568.944       33.520665
14489.279       34.701045
                 ...     
20194.323    16736.462226
9182.441     16738.158823
28204.551    16739.808353
19609.719    17615.563266
29788.629    18388.949901
Name: days_employed, Length: 19344, dtype: float64

Lo mas sensato en este caso, seria pensar que  a mayor cantidad de dias laborados, mayor es la cantidad de ingresos totales que maneja el cliente, sin embargo talvez la mediana no estamos usando la columna ni el estadistico descriptico correcto. Asi que vamos a intentarlo de otra manera.


In [ ]:
credit_scoring.groupby([credit_scoring['dob_years_group'], credit_scoring['income_type']])['total_income'].mean()

dob_years_group  income_type  
+60              business         32185.858797
                 civil servant    29465.423241
                 employee         27291.949676
                 retiree          21342.015435
20-30            business         28706.606320
                 civil servant    25101.382568
                 employee         24125.631817
                 entrepreneur     79866.103000
                 retiree          20038.393000
                 student          15712.260000
30-39            business         33145.949183
                 civil servant    27921.836553
                 employee         26191.716908
                 retiree          23122.709862
40-49            business         33989.836505
                 civil servant    28568.272491
                 employee         26193.926281
                 retiree          27020.126339
50-59            business         32385.032725
                 civil servant    25838.105730
                 employee    

En este aspecto lo que debemos tomar en cuenta ya  es que  el objetivo es completar los valores ausentes de los ingresos mensuales, es que las personas que tienen negocios o son empleados tienen mayor catidad de ingresos, y en cuanto a su respectivo grupo de edad, parece haber concordancia, ya que una persona joven que tiene un negocio, tiene un ingreso mas alto que una persona adula que es un empleado, sin embargo tampoco tienen tantos ingresos mensuales como aquellas personas que tienen mas de 40 años y tienen un negocio. Asi que podriamos asumir que los datos se comportan bien y van acorde a nuestras modificaiones.

Obtenemos un promedio general de nuestros datos ahora para verificar como se ve nuestra distribucion.

In [ ]:
credit_scoring.describe()


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income,ausent_sust
count,21450.000000,19347.000000,21450.000000,21450.000000,21450.000000,21450.000000,19347.000000,19347.000000
mean,0.480513,4640.730272,43.272821,0.817063,0.973986,0.081072,26788.802865,26788.802865
std,0.756046,5355.380393,12.570987,0.548635,1.421669,0.272952,16476.068349,16476.068349
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,3306.762000,3306.762000
25%,0.000000,926.925347,33.000000,1.000000,0.000000,0.000000,16492.834000,16492.834000
50%,0.000000,2194.207271,42.000000,1.000000,0.000000,0.000000,23202.870000,23202.870000
75%,1.000000,5535.883055,53.000000,1.000000,1.000000,0.000000,32549.611000,32549.611000
max,5.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,362496.645000,362496.645000


Evaluamos los valores medios generales de cada columna para verificar si es sensato rellenar los valores ausentes con estsos datos,intentaremos diferentes maneras de reemplazar los valores nulos, sobre todo de la columna ['total_income']

In [ ]:
def rem_pm(credit_scoring):
    return credit_scoring['total_income'].fillna(credit_scoring.groupby(['dob_years_group' , 'education'])['total_income'].transform('median'))
df_filled = rem_pm(credit_scoring)

In [ ]:
credit_scoring.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,ausent_sust
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,40620.102
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,17932.802
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,23341.752
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,42820.568
4,0,14177.753002,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,25378.572


A travez de este ajuste se espera que los valores ausentes de nuestro DataFrame original no tenga valores ausentes explicitos.

In [ ]:
credit_scoring.isna().sum()

children               0
days_employed       2103
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2103
purpose                0
dob_years_group        0
ausent_sust         2103
dtype: int64

Ahora nuestro conjunto de datos no tiene valores nulos en sus filas.

Al rellenar los valores ausentes con la media de las columnas observamos que en aun asi en 2 columnas mantenemos cierta cantidad de valores ausentes,asi que vamos a hacer un remplazo si existe algun error.


Como comprobasion final, para este apartado de reemplazaremos nuestros valores nulos con nuestra mediana general del DataFrame. 
utilizando un bloque tey-except. Para antelarnos a posibles errores.

In [ ]:
try:
    def rem_pm(credit_scoring):
        return credit_scoring.fillna(credit_scoring.mean())
    df_filled = rem_pm(credit_scoring)
except:
    df_filled.fillna(0)

In [ ]:

df_filled.reset_index().head(10)

,index,total_income
0,0,40620.102
1,1,17932.802
2,2,23341.752
3,3,42820.568
4,4,25378.572
5,5,40922.170
6,6,38484.156
7,7,21731.829
8,8,15337.093
9,9,23108.150


Realizamos una comprobacion de valores asuentes y veriricamos que coisidan con los los datos de nuestro Dtaframe original.

In [ ]:
df_filled.isna().sum()

0

In [ ]:
credit_scoring.isna().sum()

children               0
days_employed       2103
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2103
purpose                0
dob_years_group        0
ausent_sust         2103
dtype: int64

Una vez reemplazados los valores susentes con la media general de nuestros datos, comprobamos que los valores ausentes ya no existen ni en nuestra lista, y al guardarla en nuestra tabla original tambien se han eliminado de esta. Esto es util ya que podremos revertir este ajuste evitando guardar esta funsion en nuestros datos originales en el futuro, solo en el caso de ser necesario.

###  Restaurar valores en `days_employed`

Para los valores ausentes de esta columna vamos ha hacer un analisis de los valores medios o medianos agrupandola con diferentes columnas y denotaremos cual de los dos valores, medios o medianos son los mas adecuados para rellenar los valores de los dias laborados ['days_employed'].

In [ ]:
credit_scoring.info(9)

<class 'pandas.core.frame.DataFrame'>
Index: 21450 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21450 non-null  int64  
 1   days_employed     19347 non-null  float64
 2   dob_years         21450 non-null  int64  
 3   education         21450 non-null  object 
 4   education_id      21450 non-null  int64  
 5   family_status     21450 non-null  object 
 6   family_status_id  21450 non-null  int64  
 7   gender            21450 non-null  object 
 8   income_type       21450 non-null  object 
 9   debt              21450 non-null  int64  
 10  total_income      19347 non-null  float64
 11  purpose           21450 non-null  object 
 12  dob_years_group   21450 non-null  object 
 13  ausent_sust       19347 non-null  float64
dtypes: float64(3), int64(5), object(6)
memory usage: 2.5+ MB


In [ ]:
credit_scoring.groupby('income_type')['days_employed'].median()


income_type
business          1546.333214
civil servant     2689.368353
employee          1574.202821
entrepreneur       520.848083
retiree          15217.221094
student            578.751554
Name: days_employed, dtype: float64

Mediante esta agrupacion observamos que aquellas personas que tienen su tipo de ingreso a partir de un negocio, tienen mayor cantidad de dias laborados que un cliente de la misma edad pero que es empleado, por ende los clientes con un negocio tienen mayor cantidad de ingresos que los clientes que son empleados.

In [ ]:
credit_scoring.groupby('days_employed')['dob_years'].mean().sort_values()

days_employed
3533.403832      0.0
1015.395451      0.0
223.709545       0.0
4774.343347      0.0
2356.250615      0.0
                ... 
15535.879332    74.0
1729.632531     74.0
15839.599460    74.0
6682.867814     74.0
1678.969771     75.0
Name: dob_years, Length: 19347, dtype: float64

Las media y medianas de las relaciones de los dias laborados, pueden tener una diferencia importante respecto a mis datos originales. haremos una agrupacion mas detallada. utilizando nuestra columna ['dob_year_group'] para observar relaciones de los dias laborados con los grupos de edad.

In [ ]:
credit_scoring.groupby('dob_years_group')['days_employed'].mean()

dob_years_group
+60      12743.739504
20-30     1346.145405
30-39     2101.125130
40-49     3063.193788
50-59     7541.593571
Name: days_employed, dtype: float64

realizando diferentes relaciones en promedio, notamos que al grupar los dias laborados por el grupo de edad, el promedio no difiere mucho respecto a los datos origuinales,asi que podremos utlizarla para rellenar los valores ausentes en my columna.

In [ ]:
def rem_pm(credit_scoring):
    return credit_scoring['days_employed'].fillna(credit_scoring.groupby(['dob_years_group' , 'days_employed'])['days_employed'].transform('mean'))
df_sust_d = rem_pm(credit_scoring)

In [ ]:
df_sust_d

0         8437.673028
1         4024.803754
2         5623.422610
3         4124.747207
4        14177.753002
             ...     
21520     4529.316663
21521    14330.725172
21522     2113.346888
21523     3112.481705
21524     1984.507589
Name: days_employed, Length: 21450, dtype: float64

In [ ]:
credit_scoring['income_type'].fillna(df_sust_d)

0        employee
1        employee
2        employee
3        employee
4         retiree
           ...   
21520    business
21521     retiree
21522    employee
21523    employee
21524    employee
Name: income_type, Length: 21450, dtype: object

In [ ]:
credit_scoring['income_type'].isna().sum()

0

Remplazamos los valores ausentes de 'income_type' con los valores medios de la agrupacion entre las columnas del grupo de edad y los dias laborados, ya que el tipo de ingreso que un cliente tiene al mes, esta relacionado de manera creciente con su edad y sus dias laborados.

In [ ]:
credit_scoring.dropna().isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dob_years_group     0
ausent_sust         0
dtype: int64

## Clasificación de datos

Para poder responder a las preguntas y probar las diferentes hipótesis, vamos a trabajar con datos clasificados. de acuerdo a nuestras preguntas base vamos a clasificar columnas de variables categoricas, asi como numericas con el objetivo de encontrar datos relativamente parecidos que se puedan resumir en un mismpo grupo sin afectar nuestra columna original.en este caso ya hemos realizado una clasificasion en la columna de la edad de los clientes, sin embargo las columnas de texto, no las hemos clasificado aun.

En este apartado nos concentraremos en clasificar las columnas de texto y numericas que requieran una examinacion minuciosa o que tengan datos desmasiado parecidos que puedan ser mejor entendibles a travez de una columna que los clasifique en un mismo grupo.

In [ ]:
clasf1 = ['education','family_status','purpose', 'gender', 'income_type']   

In [ ]:
for column in clasf1:
    print()
    print(credit_scoring[column].value_counts())


education
secondary education    15170
bachelor's degree       5249
some college             743
primary education        282
graduate degree            6
Name: count, dtype: int64

family_status
married              12337
civil partnership     4149
unmarried             2810
divorced              1195
widow / widower        959
Name: count, dtype: int64

purpose
wedding ceremony                            791
having a wedding                            768
to have a wedding                           765
real estate transactions                    675
buy commercial real estate                  661
housing transactions                        652
transactions with commercial real estate    650
buying property for renting out             650
purchase of the house                       646
housing                                     646
purchase of the house for my family         638
construction of own property                635
property                                    633
transacti

Observamos una distribucion exagerada en la columna ['purpose']  de nuestras variables categoricas, asi que vamos a crear una columna que separe por grupos esta columna. Pero antes de ellos vamos ha realizar el mssmo proceso con las columnas numericas de my tabla

In [ ]:
clasf2 = ['children','days_employed','debt', 'total_income']   

In [ ]:
for column in clasf2:
    print()
    print(credit_scoring[column].describe())


count    21450.000000
mean         0.480513
std          0.756046
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          5.000000
Name: children, dtype: float64

count    19347.000000
mean      4640.730272
std       5355.380393
min         24.141633
25%        926.925347
50%       2194.207271
75%       5535.883055
max      18388.949901
Name: days_employed, dtype: float64

count    21450.000000
mean         0.081072
std          0.272952
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: debt, dtype: float64

count     19347.000000
mean      26788.802865
std       16476.068349
min        3306.762000
25%       16492.834000
50%       23202.870000
75%       32549.611000
max      362496.645000
Name: total_income, dtype: float64


En las columnas cuyo resultado son variables numericas,se considera mas prudente realizar una descripcion, lo que nos indica es que  no tenemos muchos inconvenientes en su distribucion, y tomando en cuenta que ya se creo una columna que agrupo a las personas por grupo de edad pues no deberiamos hacer por el momento otra modificasion ya que no hay razon alguna hasta el monento,para hacerla.

Tambien vamos ha hacer una comprovacion de valores unicos para ver que tan relevantes son en mis columnas

In [ ]:
clasf1 = ['education','family_status','purpose', 'gender', 'income_type']   
for column in clasf1:
    print()
    print(credit_scoring[column].unique())


["bachelor's degree" 'secondary education' 'some college'
 'primary education' 'graduate degree']

['married' 'civil partnership' 'widow / widower' 'divorced' 'unmarried']

['purchase of the house' 'car purchase' 'supplementary education'
 'to have a wedding' 'housing transactions' 'education' 'having a wedding'
 'purchase of the house for my family' 'buy real estate'
 'buy commercial real estate' 'buy residential real estate'
 'construction of own property' 'property' 'building a property'
 'buying a second-hand car' 'buying my own car'
 'transactions with commercial real estate' 'building a real estate'
 'housing' 'transactions with my real estate' 'cars' 'to become educated'
 'second-hand car purchase' 'getting an education' 'car'
 'wedding ceremony' 'to get a supplementary education'
 'purchase of my own house' 'real estate transactions'
 'getting higher education' 'to own a car' 'purchase of a car'
 'profile education' 'university education'
 'buying property for renting out' 'to

Al denotar los valores unicos de cada una de las columnas que contienen variables categoricas, observamos una cantidad
exageradas de este tipo de valors en la columna 'prupose', vamos a explorar una solucion para esta columna.

In [ ]:
clasf2 = ['children','days_employed','debt', 'total_income']  
for column in clasf2:
    print()
    print(credit_scoring[column].unique())


[1 0 3 2 4 5]

[8437.67302776 4024.80375385 5623.42261023 ... 2113.3468877  3112.4817052
 1984.50758853]

[0 1]

[40620.102 17932.802 23341.752 ... 14347.61  39054.888 13127.587]


En cunato a los valores unicos de las columnas numericas, no tenemos una cantidad relevante de ellos en estas columnas asi que no vamos a clasificar en este tipo de variables


La clasificasion que haremos sera para variables categoricas, completando la columna 'prupose' que contiene mayor cantidad de valores unicos. Atravez de una funcion.

In [ ]:
def category_rt(purpose):
    purpose = purpose.lower()
    
    if 'wedding' in purpose:
        return 'wedding'
    elif 'car' in purpose:
        return 'car_p'
    elif 'real estate' in purpose:
        return 'real_state'
    elif 'house' in purpose or 'housing' in purpose or 'property' in purpose:
        return 'rw_home'
    elif 'education' in purpose or 'educated' in purpose or 'university' in purpose:
        return 'education'
    else:
        return 'other'

Nuestra funsion definida, ecuentra palabras clave en los valores unicos de la columna que indica el proposito del cliente para adquirir un prestamo ['purpose'], con el objetivo de agruparlos en una sola categoria.Vamos aplicarla para verificar el resultado.

In [ ]:
credit_scoring['purpose'].apply(category_rt)

0          rw_home
1            car_p
2          rw_home
3        education
4          wedding
           ...    
21520      rw_home
21521        car_p
21522      rw_home
21523        car_p
21524        car_p
Name: purpose, Length: 21450, dtype: object

Nuestra columna se ve bien, vamos a aplicarla en nuestros datos originales para evaluar el resultado de sus relaciones con otras columnas.

In [ ]:
credit_scoring['category_rt'] = credit_scoring['purpose'].apply(category_rt)

In [ ]:
credit_scoring.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_years_group,ausent_sust,category_rt
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49,40620.102,rw_home
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39,17932.802,car_p
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39,23341.752,rw_home
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39,42820.568,education
4,0,14177.753002,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59,25378.572,wedding


Una vez que tenemos nuestra tabla modificada, realizaremos una relacion la columna de mide el incumplimineto de pago a tiempo del prestamo, ['debt'] con el objetivo de averiguar cual proposito tiene mayor posiblidad de tener un pago icumplido a tiempo.

In [ ]:
credit_scoring.groupby('category_rt')['debt'].mean().sort_values()

category_rt
rw_home       0.070134
real_state    0.075286
wedding       0.080034
education     0.092200
car_p         0.093380
Name: debt, dtype: float64

Esta relacion es importante, ya nos  muestra un promedio para cada grupo de categoria. A travez de ella observamos que, a pesar de ser una cantidad minima. Es mas probable que aquellos clientes cuyo proposito principal para solicitar su prestamo es, educativo, o la adquisicion de un vehiculo tenga influencia en la cancelacion a tiempo de este prestamo.

## Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [ ]:
credit_scoring.groupby('children')['debt'].median().sort_values()

children
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
Name: debt, dtype: float64

Para este apartado haremos dos relaciones. La  la columna ['children'],para averiguar si tiene alguna relacion especifica entre el numero de hijos del cliente y el incumplimineto del pago a tiempo de su prestamo.

**Conclusión**

Una vez que hacemos una relacion entre el numero de hijos  y los pagos a tiempo se concluye que:

*En promedio no existe una relacion muy espesifica entre el numero de hijos y el hecho de haber pagado a tiempo una deuda

*El hecho de que una persona tenga hijos no tiene relacion, segun el analisis previo, en el incumplimiento de pagos de una prestamos

*Quizas exista un porcentaje de personas que si tengan pagos incumplidos, pero el numero de hijos o si los tiene o no, no es un factor demasiado relevante para afirmar que si influye el numero de hijos en tal icumplimiento de un pago a tiempo.


**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [ ]:
credit_scoring.groupby('family_status')['debt'].mean().sort_values()

family_status
widow / widower      0.065693
divorced             0.071130
married              0.075302
civil partnership    0.093517
unmarried            0.097509
Name: debt, dtype: float64

**Conclusión**

Relacionando el estatus de familia ['family_status'] con el incumplimiento de un pago ['debt'] se concluye lo siguiente:

*En promedio el estatus de familia no tiene ninguna incontingencia para el incumplimiento de un pago a tiempo,ya que no representa un numero relevante de clientes cuyo estatus de familia le impida cumplir sus respectivos pagos

*Se descarta la posibilidad de que el incumplimiento de un pago, este ligado enteramente a la situasion familiar.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [ ]:
credit_scoring.groupby('total_income')['debt'].mean().sort_values()

total_income
23200.877    0.0
28216.603    0.0
28209.905    0.0
28209.807    0.0
28204.551    0.0
            ... 
26268.045    1.0
13098.098    1.0
40767.448    1.0
13125.048    1.0
3306.762     1.0
Name: debt, Length: 19344, dtype: float64

**Conclusión**

Despues de realcionar el nivel de ingresos de cada cliente ['total_income'] con el incumplimiento de un pago a tiempo ['debt']. Podriamos concluir los singuiente:

*En esta columna si econtramos una coorrelacion entre los clientes que tienen un nivel de ingresos bajos y el incumplimineto de un pago a tiempo.

*Los clientes que en promedio manejan un nivel de igresos bajos, si han tenido inconvenientes en el pago a tiempo de sus deudas, asi que el nivel de ingresos en este caso si representa un factor de riesgo para dar una buena calificasion a este tipo de clientes


**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [ ]:
credit_scoring.groupby(['category_rt'])['debt'].mean().sort_values()

category_rt
rw_home       0.070134
real_state    0.075286
wedding       0.080034
education     0.092200
car_p         0.093380
Name: debt, dtype: float64

**Conclusión**

Vamos a utilizar la columna ['category_rt'], para menejar resultados mas generales. Y en promedio, los clientes que tienen un  incumplimineto del pago a tiempo ['debt'] se ven  mayormente reflejado en el prestamos cuyo proposito comprende fines educativos o en la compra o venta de automoviles.

# Conclusión general 

Evaluar correctamente los fatores que puedan influir en el incumplimieto de un pago a tiempo de un prestamo, no es tarea facil, y mas si se requiere un grado de exactitud  relevante para  dar una calificasion  especifica al cliente.
Este dataframe ha representado un reto para my, sin embargo,se logro manejar de manera adecuada los valores ausentes,duplicados como tambien los valores unicos, tratando de no afectar en lo mas minimo los datos originales del dataframe, ulizando medianas y medias de los mismos datos para rellenar los valores ausentes y utilizando columnas diferentes en base a nuestras columnas originales, para no afectar los datos originales de las mismas, en cuanto a valores unicos, no se realizo muchas modificasiones ya que son importantes para nuestra tabla general.  Concluyendo lo siguiente:

*Tanto el numero de hijos como el estatus de familia no tienen un porcentanje en promedio relevante que afecte al incumplimineto del pago de un prestamo a tiempo, ya que los promedios no reflejan que el cliente haya incumplido alguna vez un prestamo, idepentientemente si tiene o no tiene hijos o si tiene un alto estatus familiar.


*La agrupacion de clientes por rango de edad, resulta muy conveniente al momento de averiguar el tipo de ingresosos mensuales que estos podrian tener.


*En promedio,el nivel de ingresos de un cliente si afecta significativamente el icumplimineto del pago de un prestamo ya que aquellos clientes cuyo ingreso mensual es mas bajo  tienen  incumplidos al menos 1 pago a tiempo.


*Aquellas personas cuyo proposito de prestamos son fines educativos o de compra y venta de bienes, tienen mayor probabilidad de incumplir en el pago de su prestamo ya que en promedio este tipo de propositos tienen algun incumplimineto de pago a tiempo en su prestamo.


*Para finalizar se espera que este analisis se tome muy en cuenta al momento de calificar al cliente,segun my analisis solamente el nivel de ingresos de cada cliente representa una relevancia en el incumplimineto de un pago a tiempo.